In [1]:
import tensorflow as tf
print(tf.__version__)

for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

2.0.0


#### NiN块是NiN中的基础块。它由一个卷积层加两个充当全连接层的1×11×1卷积层串联而成。其中第一个卷积层的超参数可以自行设置，而第二和第三个卷积层的超参数一般是固定的。

In [2]:
# 定义一个block
def nin_block(num_channels, kernel_size, strides, padding):
    blk = tf.keras.models.Sequential()
    blk.add(tf.keras.layers.Conv2D(num_channels, kernel_size,
                                   strides=strides, padding=padding, activation='relu',kernel_initializer='glorot_uniform')) 
    blk.add(tf.keras.layers.Conv2D(num_channels, kernel_size=1,activation='relu',kernel_initializer='glorot_uniform')) 
    blk.add(tf.keras.layers.Conv2D(num_channels, kernel_size=1,activation='relu',kernel_initializer='glorot_uniform'))    
    return blk

#### NiN去掉了AlexNet最后的3个全连接层，取而代之地，NiN使用了输出通道数等于标签类别数的NiN块，然后使用全局平均池化层对每个通道中所有元素求平均并直接用于分类。

In [3]:
net = tf.keras.models.Sequential()
net.add(nin_block(96, kernel_size=11, strides=4, padding='valid'))
net.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=2))
net.add(nin_block(256, kernel_size=5, strides=1, padding='same'))
net.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=2))
net.add(nin_block(384, kernel_size=3, strides=1, padding='same'))
net.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=2))
net.add(tf.keras.layers.Dropout(0.5))
net.add(nin_block(10, kernel_size=3, strides=1, padding='same'))
net.add(tf.keras.layers.GlobalAveragePooling2D())
net.add(tf.keras.layers.Flatten())


In [4]:
X = tf.random.uniform((1,224,224,1))
for blk in net.layers:
    X = blk(X)
    print(blk.name, 'output shape:\t', X.shape)


sequential_1 output shape:	 (1, 54, 54, 96)
max_pooling2d output shape:	 (1, 26, 26, 96)
sequential_2 output shape:	 (1, 26, 26, 256)
max_pooling2d_1 output shape:	 (1, 12, 12, 256)
sequential_3 output shape:	 (1, 12, 12, 384)
max_pooling2d_2 output shape:	 (1, 5, 5, 384)
dropout output shape:	 (1, 5, 5, 384)
sequential_4 output shape:	 (1, 5, 5, 10)
global_average_pooling2d output shape:	 (1, 10)
flatten output shape:	 (1, 10)


In [5]:
# 获取数据
from tensorflow.keras.datasets import fashion_mnist
import matplotlib.pyplot as plt


(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()


# 数据预处理
def data_scale(x, y):
    x = tf.cast(x, tf.float32)
    x = x / 255.0
    x = tf.reshape(x, (x.shape[0], x.shape[1], 1))
    x = tf.image.resize_with_pad(image=x, target_height=224,target_width=224)
    
    return x, y
# 由于笔记本训练太慢了，使用1000条数据，跑一下先,算力够的可以直接使用全部数据更加明显
train_db = tf.data.Dataset.from_tensor_slices((x_train[0:5000],y_train[0:5000])).shuffle(20).map(data_scale).batch(32)
test_db = tf.data.Dataset.from_tensor_slices((x_test[0:1000],y_test[0:1000])).shuffle(20).map(data_scale).batch(32)

In [6]:
# 定义优化器和损失函数
optimizer = tf.keras.optimizers.SGD(lr=1e-2)
loss = tf.keras.losses.sparse_categorical_crossentropy
net.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
net.fit_generator(train_db, epochs=1, validation_data=test_db)    # 这里就不跑太多轮了，有机器可以自己调参跑个好的结果
net.summary()

129/157 [=======================>......] - ETA: 1:58 - loss: 9.6310 - accuracy: 0.0995

In [ ]:
# 可以像alexnet一样，打印中间特征层看一下
X = next(iter(train_db))[0][0]

def show(X):
    X_ = tf.squeeze(X)
    plt.imshow(X_)
    plt.figure(figsize=(5,5))
    plt.show()


X = tf.expand_dims(X, axis=0)
# 打印前 8 层的部分特征图
for blk in net.layers[0:8]:
    print(blk.name,'itput shape:\t',X.shape)
    show(X[0,:,:,0])
    X = blk(X)
    print(blk.name, 'output shape:\t', X.shape)
    for i in range(3):
        show(X[0,:,:,i])